# Mongolia Lineaments #
Using [geemap](https://geemap.org/) to analyze lineaments generated from Landsat 7 in Mongolia using [Canny Edge Detector](https://developers.google.com/earth-engine/apidocs/ee-algorithms-cannyedgedetector?hl=en) and [Hough Transform](https://developers.google.com/earth-engine/apidocs/ee-algorithms-houghtransform?hl=en).

In [3]:
# Import earth engine and geemap

import ee
import geemap

ee.Initialize()
#############Import Shapefiles###########
mongoShape = ee.FeatureCollection("users/pirrie/MongoliaAssets/MongoliaOutline")
potSuture = ee.FeatureCollection("users/pirrie/MongoliaAssets/PotentialSuture")
mongoGeom = mongoShape.geometry()

Map = geemap.Map(center=[46.935,103.376], zoom=5)

# Add Mongolia Shape, Suture
Map.addLayer(mongoShape, {'color': '000000'}, 'Mongolia', True, 0.5)
Map.addLayer(potSuture, {'color': 'FF0000'}, 'Suture (Zorin)', False, 0.5)

# Load Landsat 7 data, filter by date and bounds.
collection = ee.ImageCollection("LANDSAT/LE07/C01/T1") \
  .filterDate('2019-01-01', '2021-08-10') \
  .filter(ee.Filter.calendarRange(6, 9, 'month'))

# Also filter the collection by the IMAGE_QUALITY property.
filtered = collection \
  .filterMetadata('IMAGE_QUALITY', 'equals', 9)

# Create two composites to check the effect of filtering by IMAGE_QUALITY.
goodComposite = ee.Algorithms.Landsat.simpleComposite(filtered, 75, 3)

# Add PC band
Panchromatic = goodComposite \
  .clip(mongoShape) \
  .select(['B8'])

Map.addLayer(Panchromatic,
             {'gain': 1.5},
             'Panchromatic',
             False)

#############Edge Detection###########

# Detect edges in the panchromatic composite. Variables are (image, threshold, sigma)
canny = ee.Algorithms.CannyEdgeDetector(Panchromatic, 10, 1.5)

# Mask the image with itself to get rid of areas with no edges.
canny = canny \
  .updateMask(canny)

Map.addLayer(canny, {'min': 0, 'max': 1, 'palette': 'FF0000'}, 'CE Pan', False)

#############Hough Transformation###########

# Perform Hough transform of the Canny result and display.
hough = ee.Algorithms.HoughTransform(canny, 256, 64, 300, True)

Map.addLayer(hough, {'palette': 'FFFFFF'}, 'hough', True, 0.5)

Map

Map(center=[46.935, 103.376], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…